In [ ]:
!pip uninstall -y torch # Uninstall current torch version to avoid potential conflicts
!pip install torch==2.0.1+cu118 --index-url https://download.pytorch.org/whl/cu118  # Reinstall specific torch version, specify CUDA version if needed
!pip install vllm transformers

Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp311-cp311-linux_x86_64.whl (2267.3 MB)


In [ ]:
import vllm
import json
import os
from transformers import AutoModelForSeq2SeqLM, AutoConfig, AutoTokenizer

# --- Turing Tumble Hybrid Neuro-Symbolic AI ---

# ... (Your existing code for imports, image upload, VLM caption extraction,
# logic graph construction, PyTorch Geometric conversion, serialization,
# LLM planning prompt, GNN model, and pipeline) ...


# --- vLLM Integration with Hugging Face Model ---

# Specify the Hugging Face model identifier
model_name = "google/flan-t5-xl"  # Example: Replace with your desired model

# Create the local model directory if it doesn't exist
model_dir = "model_cache"
os.makedirs(model_dir, exist_ok=True)

# Download and cache the model configuration and weights
config = AutoConfig.from_pretrained(model_name, cache_dir=model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config, cache_dir=model_dir)

# Download and cache the tokenizer (if needed)
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_dir)

# Set the model path to the cached directory + config file
# This assumes the config file name is config.json
model_path = os.path.join(model_dir, "config.json")

# Initialize vLLM with the model directory (model_dir) instead of the config file path
llm = vllm.LLM(model=model_dir)


# --- Modified plan_with_llm function ---

def plan_with_llm(board_json):
    prompt = generate_llm_prompt(board_json)

    # Generate response using vLLM
    outputs = llm.generate([prompt],
                            sampling_params=vllm.SamplingParams(temperature=0.7, max_tokens=128))

    # Extract the generated text
    response_text = outputs[0].outputs[0].text

    # Assuming the response is JSON, parse it
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        print("Warning: LLM response is not valid JSON. Returning empty action.")
        return {}  # Return an empty action if JSON parsing fails

# ... (Rest of your code, including hybrid_pipeline and running the pipeline) ...


In [ ]:
# Turing Tumble Hybrid Neuro-Symbolic AI (Colab Notebook Version)

# ------------------------
# Install Dependencies (Colab only)
# ------------------------
!pip install -q transformers torch torchvision torch-geometric networkx openai Pillow
!pip install -q openai
# ------------------------
# Imports
# ------------------------
import torch
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from torch_geometric.data import Data as GNNData
from torch_geometric.nn import GCNConv
import networkx as nx
from PIL import Image
import json
import openai
import matplotlib.pyplot as plt
import os # Import the os module

from openai import OpenAI
client = OpenAI()

# Set your OpenAI API key using an environment variable
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"  # Replace YOUR_API_KEY with your actual API key


response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

# ------------------------
# Upload Image (Colab)
# ------------------------
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# ------------------------
# VLM Caption Extraction (BLIP)
# ------------------------
def extract_board_description_with_blip(image_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# ------------------------
# Dummy Logic Graph Construction
# ------------------------
def build_dummy_graph_from_caption(caption):
    G = nx.DiGraph()
    G.add_node(0, label="ball", position=[0, 0])
    G.add_node(1, label="ramp", position=[1, 0], orientation="right")
    G.add_node(2, label="gear", position=[2, 1])
    G.add_edges_from([(0, 1), (1, 2)])
    return G

# ------------------------
# Convert to PyTorch Geometric Format
# ------------------------
def convert_nx_to_gnn_data(graph):
    node_labels = [ord(graph.nodes[n]['label'][0]) for n in graph.nodes()]
    x = torch.tensor([[l] for l in node_labels], dtype=torch.float)
    edge_index = torch.tensor(list(graph.edges())).t().contiguous()
    return GNNData(x=x, edge_index=edge_index)

# ------------------------
# Serialize for LLM
# ------------------------
def serialize_board_to_json(graph):
    nodes = []
    for nid, attrs in graph.nodes(data=True):
        nodes.append({
            "id": nid,
            "type": attrs.get("label", "unknown"),
            "position": attrs.get("position", [0, 0]),
            "orientation": attrs.get("orientation", "none")
        })
    edges = [{"from": u, "to": v} for u, v in graph.edges()]
    return {
        "nodes": nodes,
        "connections": edges,
        "marbles": [],
        "gear_states": [],
        "bit_states": [],
        "goal": "Trigger final bit to ON using blue marble"
    }

# ------------------------
# LLM Planning Prompt
# ------------------------
def generate_llm_prompt(board_json):
    return f"""
You are an AI agent acting as a symbolic planner for a Turing Tumble puzzle.
Your goal is: \"{board_json['goal']}\"

Here is the current board state:
{json.dumps(board_json, indent=2)}

Reply with the best next action using this format:
{{
  \"action\": \"place_component\",
  \"component\": \"ramp\",
  \"position\": [4, 5],
  \"orientation\": \"left\"
}}

Only use these actions:
- \"place_component\"
- \"remove_component\"
- \"launch_marble\"
"""

def plan_with_llm(board_json):
    prompt = generate_llm_prompt(board_json)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return json.loads(response["choices"][0]["message"]["content"])

# ------------------------
# Simple GNN Model
# ------------------------
class TuringTumbleGNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# ------------------------
# Pipeline
# ------------------------
def hybrid_pipeline(image_path):
    print("[1] Extracting board info with VLM...")
    caption = extract_board_description_with_blip(image_path)
    print("\n[Caption]:", caption)

    print("[2] Building logic graph from caption...")
    nx_graph = build_dummy_graph_from_caption(caption)

    print("[3] Serializing for LLM planner...")
    board_json = serialize_board_to_json(nx_graph)

    print("[4] Invoking LLM planner...")
    action = plan_with_llm(board_json)
    print("\n[LLM Suggested Action]:", action)

    print("[5] Symbolic reasoning with GNN...")
    gnn_data = convert_nx_to_gnn_data(nx_graph)
    model = TuringTumbleGNN()
    output = model(gnn_data)
    print("\n[GNN Output]:", output)

    return output, action

# ------------------------
# Run Pipeline
# ------------------------
output, action = hybrid_pipeline(image_path)